In [136]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as  plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [52]:
dataset=pd.read_csv("/content/data/diabetes.csv")

EDA

In [53]:
dataset.head(10)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


In [54]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


Dataset has no null or categorical values. Small dataset,might be prone to overfitting

In [55]:
dataset.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


Dataset ranges from values as small 0.34 to 120. Normalization must be done

In [56]:
dataset["Outcome"].value_counts()


Outcome
0    500
1    268
Name: count, dtype: int64

Imbalanced class, Non diabetic is the majority class

Binning the Glucose category in order to perform stratified samp

In [66]:
labels=[1,2,3,4]
bins=[-1,50,100,150,np.inf]

In [67]:
dataset['Glucose_cat']=pd.cut(dataset["Glucose"],labels=labels,bins=bins)

In [68]:
dataset["Glucose"].replace(0,0.001,inplace=True)

In [69]:
dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Glucose_cat
0,6,148.0,72,35,0,33.6,0.627,50,1,3
1,1,85.0,66,29,0,26.6,0.351,31,0,2
2,8,183.0,64,0,0,23.3,0.672,32,1,4
3,1,89.0,66,23,94,28.1,0.167,21,0,2
4,0,137.0,40,35,168,43.1,2.288,33,1,3


In [70]:
X=dataset.drop(columns="Outcome")
y=dataset["Outcome"]

In [71]:
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Glucose_cat
0,6,148.0,72,35,0,33.6,0.627,50,3
1,1,85.0,66,29,0,26.6,0.351,31,2
2,8,183.0,64,0,0,23.3,0.672,32,4
3,1,89.0,66,23,94,28.1,0.167,21,2
4,0,137.0,40,35,168,43.1,2.288,33,3


In [72]:
y

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

In [73]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=dataset["Glucose_cat"],shuffle=True)

In [74]:
X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Glucose_cat
566,1,99.0,72,30,18,38.6,0.412,21,2
703,2,129.0,0,0,0,38.5,0.304,41,3
73,4,129.0,86,20,270,35.1,0.231,23,3
46,1,146.0,56,0,0,29.7,0.564,29,3
643,4,90.0,0,0,0,28.0,0.610,31,2


In [75]:
X_test.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Glucose_cat
346,1,139.000,46,19,83,28.7,0.654,22,3
365,5,99.000,54,28,83,34.0,0.499,30,2
451,2,134.000,70,0,0,28.9,0.542,23,3
737,8,65.000,72,23,0,32.0,0.600,42,2
342,1,0.001,68,35,0,32.0,0.389,22,1


Dropping glucose category since it is redundant after stratified sampling

In [79]:
X_train.drop(columns="Glucose_cat",inplace=True)


KeyError: "['Glucose_cat'] not found in axis"

In [78]:
X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
566,1,99.0,72,30,18,38.6,0.412,21
703,2,129.0,0,0,0,38.5,0.304,41
73,4,129.0,86,20,270,35.1,0.231,23
46,1,146.0,56,0,0,29.7,0.564,29
643,4,90.0,0,0,0,28.0,0.610,31


In [80]:
X_test.drop(columns="Glucose_cat",inplace=True)

Scaling data

In [83]:
scaler=StandardScaler()
cols=X_train.columns
X_train=scaler.fit_transform(X_train)

In [85]:
X_train=pd.DataFrame(X_train,columns=cols)


In [86]:
X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,-0.837633,-0.692381,0.132757,0.564997,-0.541934,0.834743,-0.171503,-1.037120
1,-0.536164,0.241405,-3.648766,-1.279279,-0.697653,0.821948,-0.501137,0.677220
2,0.066775,0.241405,0.868053,-0.049761,1.638143,0.386904,-0.723945,-0.865686
3,-0.837633,0.770551,-0.707581,-1.279279,-0.697653,-0.304047,0.292426,-0.351384
4,0.066775,-0.972517,-3.648766,-1.279279,-0.697653,-0.521568,0.432826,-0.179950


In [87]:
X_test=pd.DataFrame(scaler.fit_transform(X_test),columns=cols)

In [88]:
X_test.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,-0.876541,0.624932,-1.058724,-0.030848,0.057890,-0.364442,0.486382,-0.977741
1,0.250702,-0.656980,-0.667380,0.588794,0.057890,0.288306,0.036105,-0.315157
2,-0.594731,0.464693,0.115307,-1.338981,-0.674012,-0.339810,0.161021,-0.894918
3,1.096134,-1.746606,0.213142,0.244548,-0.674012,0.041986,0.329511,0.678718
4,-0.876541,-3.829681,0.017471,1.070738,-0.674012,0.041986,-0.283446,-0.977741


Training models

In [89]:
metrics=["accuracy","precision","recall"]

Logistic Regression

In [95]:
logreg=LogisticRegression()
cross_val_scores=cross_validate(logreg,X=X_train,y=y_train,cv=10,scoring=metrics)

In [96]:
cross_val_scores

{'fit_time': array([0.02120662, 0.00928378, 0.00875807, 0.00752425, 0.00574327,
        0.00726485, 0.0050807 , 0.00507951, 0.00514078, 0.00541162]),
 'score_time': array([0.01022935, 0.0100491 , 0.00966597, 0.00587106, 0.00742412,
        0.00744057, 0.00536036, 0.00536346, 0.00578141, 0.00535679]),
 'test_accuracy': array([0.72580645, 0.79032258, 0.72580645, 0.70967742, 0.75409836,
        0.86885246, 0.67213115, 0.75409836, 0.78688525, 0.75409836]),
 'test_precision': array([0.64705882, 0.8       , 0.69230769, 0.6       , 0.6875    ,
        0.80952381, 0.53333333, 0.73333333, 0.76470588, 0.70588235]),
 'test_recall': array([0.5       , 0.54545455, 0.40909091, 0.54545455, 0.52380952,
        0.80952381, 0.38095238, 0.5       , 0.59090909, 0.54545455])}

In [113]:
for metric in metrics:
  print (f"Mean {metric.capitalize()} :" )
  print(np.mean(cross_val_scores[f"test_{metric}"]))

Mean Accuracy :
0.7541776837652036
Mean Precision :
0.6973645227321698
Mean Recall :
0.535064935064935


In [116]:
rf=RandomForestClassifier()
cross_val_scores=cross_validate(rf,X=X_train,y=y_train,cv=10,scoring=metrics)



In [117]:
for metric in metrics:
  print (f"Mean {metric.capitalize()} :" )
  print(np.mean(cross_val_scores[f"test_{metric}"]))

Mean Accuracy :
0.7589635113696458
Mean Precision :
0.6930120030468327
Mean Recall :
0.5902597402597403


In [129]:
kn_clf=KNeighborsClassifier(n_neighbors=10)
cross_val_scores=cross_validate(kn_clf,X=X_train,y=y_train,cv=10,scoring=metrics)


In [130]:
for metric in metrics:
  print (f"Mean {metric.capitalize()} :" )
  print(np.mean(cross_val_scores[f"test_{metric}"]))

Mean Accuracy :
0.7196985721840296
Mean Precision :
0.6556787575169929
Mean Recall :
0.4324675324675325


In [122]:
svm_svc=SVC(kernel="poly",probability=True)

In [128]:
cross_val_scores=cross_validate(svm_svc,X=X_train,y=y_train,cv=10,scoring=metrics)
for metric in metrics:
  print (f"Mean {metric.capitalize()} :" )
  print(np.mean(cross_val_scores[f"test_{metric}"]))

Mean Accuracy :
0.7394764674775252
Mean Precision :
0.7936113886113886
Mean Recall :
0.3692640692640693


In [137]:
decTree=DecisionTreeClassifier()

Ensemble method

In [139]:
voting_clf=VotingClassifier(estimators=[('svm',svm_svc),('log',logreg),('knn',kn_clf),('dTree',decTree),('random_forest',rf)],voting='soft')


In [134]:
cross_val_scores=cross_validate(voting_clf,X=X_train,y=y_train,cv=10,scoring=metrics)
for metric in metrics:
  print (f"Mean {metric.capitalize()} :" )
  print(np.mean(cross_val_scores[f"test_{metric}"]))

Mean Accuracy :
0.7590428344791116
Mean Precision :
0.7228781087333719
Mean Recall :
0.5166666666666666


In [ ]:
voting_clf.scor()